# Problem Statement
An imaginary university wants to restrict its student’s entrance to the campus. Suppose that there are N entrances, M students and G guards. Due to the security measures, each student is known to be assigned a gate through which they should enter the university.

The N entrances will be numbered from 1 to N. Regarding one entry, the entrance will be opened right before the first student’s arrival and closed right after the arrival of the last student that should enter through that specific entrance. We will assume that two students can not enter the university simultaneously. For an entry to be protected, a guard should be assigned to it. Notice that a guard cannot leave his post until the door he was given is closed.

Assume that the university's head of the guards knows the order in which the students are coming to the university (yeah, they know you more than you know about yourself!). He wants you to help him if having only G guards is enough to address the restrictions they wish to apply (in other words, whether there will be a moment when more than G entrances should be opened or not).

**Input:**

In the first line, you will be given the values of N, M and G, which correspond to the number of entrances to the university, the number of students in the university and the number of guards that the head of the guards intends to use to apply these restrictions respectively.
$$1 \le N \le 10^6$$
$$1 \le M \le 10^3$$
$$1 \le G \le 100$$
In the second line, you will be given M integers which the ith integer corresponds to the entrance that has been assigned to the ith student to enter the university. Students will enter the university in the same order . 

**Ouptut:**

Output “YES” if having G guards is enough to respect the restrictions, and “NO” if it is not enough. 

**Examples:**

__Input 1__
```
4 5 1
1 1 3 3 3
```
__Output 2__
```
YES
```
In this example, we only have one guard in the university. Initially, the guard will be assigned to protect entrance 1'. After the second student arrives, the guard will close access ‘1' and go to entrance '3'. In this case, having only one guard is enough to address the restrictions. 
___

__Input 2__
```
2 5 1 
1 2 1 2 2
```
__Output 2__
```
NO
```

Ideas
* gates to guards
* pop and push guards onto a stack
* loop over student gates
* if a gate is closed, it cannot be opened again
* I must sort the students somehow because with more guards, I need to know which gates can be closed

How do I know which gates can be closed?
* If there are no more students waiting

So I need to create a line of students for each gate, and if the line is empty only then can I remove the guard.

```
- create n gates
- go over students and put each student in a queue

- find the current first student
- if gate has guard
- let student in
- if gate has no guard
- find a guard 

for each student gate
    if has guard
        go to next student
    if has no guard
        if guard left
            pop guard
            assign to new gate
        if no guard left
            remove guard from gate
```

## Read inputs


In [1]:
with open('students.txt', 'r') as file:
    n_gates, n_students, n_guards = map(int, file.readline().split())
    student_gates = list(map(int, file.readline().split()))

In [2]:
class Gate:
    def __init__(self, number):
        self.number = number
        self.closed = False
        self.students = []
        self.priority = 10**3 + 1 # 10**3 is the max number of students possible, so this gate will be last
        self.guard = None
        
    def close(self):
        self.closed = True
        
    def set_guard(self, guard):
        self.guard = guard
        
    def has_guard(self):
        return self.guard != None
        
    def remove_guard(self):
        guard = self.guard
        self.guard = None
        self.close()
        
        return guard
    
    def add_student(self, student):
        self.students.append(student)
        
    def let_student_in(self):
        self.students.pop(0)
        
    def can_be_closed(self):
        return len(self.students) == 0
    
    def set_priority(self):
        if self.students:
            self.priority = self.students[0]
        else:
            self.priority = 10**3 + 1
        
    def __lt__(self, other):
        return self.priority < other.priority
    
    def __repr__(self):
        return f'Gate; {self.number}. Closed; {self.closed}. Students waiting; {self.students}'

## Setup objects

In [3]:
guards = list(range(n_guards))

In [4]:
gates = {gate_number: Gate(gate_number) for gate_number in range(1, n_gates + 1)}

In [5]:
for student, student_gate in zip(range(n_students), student_gates):
    gates[student_gate].add_student(student)
    gates[student_gate].set_priority()

## Make sure we always let the first student enter

In [6]:
from functools import reduce

In [7]:
from queue import PriorityQueue

queue = PriorityQueue(maxsize=n_gates)
for gate in gates.values():
    queue.put((gate.priority, gate))

In [8]:
students_left = True
print(queue.queue)
while students_left:
    _, gate = queue.get()
    print(gate, guards)
    # If a gate has a guard, I want to let a student in
    if gate.has_guard():
        gate.let_student_in()
        gate.set_priority()
        
        # If this was the last student, we can remove the guard
        if gate.can_be_closed():
            guards.append(gate.remove_guard())
        
    else:
        # If a gate has no guards I need to add a free guard
        if guards:
                
            guard = guards.pop(0)
            gate.set_guard(guard)
            gate.let_student_in()
            gate.set_priority()
            
            if gate.can_be_closed():
                guards.append(gate.remove_guard())
                
             # If this was the last student, we can remove the guard
            
            
        # If there are no guards left
        else:
            print("NO", guards)
            break
            
    queue.put((gate.priority, gate))
            
    students_left = any(map(lambda gate: gate.can_be_closed(), gates.values()))
    print(list(map(lambda gate: gate.can_be_closed(), gates.values())))
            
print("YES")

[(0, Gate; 1. Closed; False. Students waiting; [0, 1]), (1001, Gate; 2. Closed; False. Students waiting; []), (2, Gate; 3. Closed; False. Students waiting; [2, 3, 4]), (1001, Gate; 4. Closed; False. Students waiting; [])]
Gate; 1. Closed; False. Students waiting; [0, 1] [0]
[False, True, False, True]
Gate; 1. Closed; False. Students waiting; [1] []
[True, True, False, True]
Gate; 3. Closed; False. Students waiting; [2, 3, 4] [0]
[True, True, False, True]
Gate; 3. Closed; False. Students waiting; [3, 4] []
[True, True, False, True]
Gate; 3. Closed; False. Students waiting; [4] []
[True, True, True, True]
Gate; 2. Closed; False. Students waiting; [] [0]


IndexError: pop from empty list